<a href="https://colab.research.google.com/github/ShinAsakawa/2019komazawa/blob/master/notebooks/2019komazawa_cifar10_exercise03.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<center>
<h1>[駒澤大学心理学特講IIIAディープラーニングの心理学的解釈](https://www.komazawa-u.ac.jp/~kyoumu/syllabus_html/detail/074401.html)</h1>
</center>

<div align='right'>
<a href='mailto:asakawa@ieee.org'>Shin Aasakawa</a>, all rights reserved.<br>
Date: 19/Apr/2019<br>
 MIT license
</div>

---

# CIFAR10 によるはじめての CNN

- CIFAR10 とは[アレックス・クリゼンスキー](http://www.cs.toronto.edu/~kriz/index.html)が管理している画像データセットです。
- CIFAR とはカナディアン インスティチュート フォー アドバンストリサーチ直訳すればカナダ先端研究所くらいの意味です。
- 10 とは 10 分類，すなわち10種類のデータを分類するデータセットであることを意味します。
- 詳しくは本家サイト  <http://www.cs.toronto.edu/~kriz/cifar.html> をご覧ください。



In [0]:
# まず必要なライブラリを輸入 import します
import numpy as np
import matplotlib.pyplot as plt
from keras.datasets import cifar10

In [0]:
# このセルでは cifar10 データセットを読み込んでいます
# 読み込んだデータセットのサイズを表示しています。
# データは 4階テンソル，すなわち4次元のデータです。
# 最初の次元がデータ数，2番めと3番めの次元は画像の縦横のピクセル数です。
# 最後の次元はチャンネル数，この場合色の次元ですから 3 (R,G,B)になります。
(x_train, y_train), (x_test, y_test) = cifar10.load_data() 
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

num_classes=10
class_names={0:'airplane', 1:'automobile', 2:'bird', 3:'cat', 4:'deer', 
             5:'dog', 6:'frog', 7:'horse', 8:'ship', 9:'truck'}

In [0]:
# 読み込んだ画像を表示してみます。何度かこのセルを実行するとその都度異なる画像を表示します。
# 乱数を使ってその都度異なる画像を表示させているためです
# 何度か繰り返してデータの性質を体感してください
fig = plt.figure(figsize=(8,3))
for i in range(num_classes):
    ax = fig.add_subplot(2, 5, 1 + i, xticks=[], yticks=[])
    idx = np.where(y_train[:]==i)[0]
    features_idx = x_train[idx,::]
    img_num = np.random.randint(features_idx.shape[0])
    im = features_idx[img_num]
    ax.set_title(class_names[i])
    plt.imshow(im)
plt.show()

In [0]:
# ディープラーニングのモデルを作成するために必要な情報を輸入 import します。
import keras
# 詳しくは 以下の情報をご覧ください
# シーケンシャル: <https://keras.io/ja/getting-started/sequential-model-guide/>
# デンス: <https://keras.io/ja/layers/core/#dense>
# ドロップアウト: <https://keras.io/ja/layers/core/#dropout>
# アクティベーション: <https://keras.io/ja/layers/advanced-activations/>
# フラッテン: <https://keras.io/ja/layers/core/#flatten>
# コンブ2D: <https://keras.io/ja/layers/convolutional/#conv2d>
# マックスプーリング2D: <https://keras.io/ja/layers/pooling/#maxpooling2d>
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D

In [0]:
model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=x_train.shape[1:]))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

# 以下のコメントを外すと一層ネットワークに追加されます。性能を比較して見るときに使います
#model.add(Conv2D(64, (3, 3), padding='same'))
#model.add(Activation('relu'))
#model.add(Conv2D(64, (3, 3)))
#model.add(Activation('relu'))
#model.add(MaxPooling2D(pool_size=(2, 2)))
#model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

# initiate RMSprop optimizer
opt = keras.optimizers.rmsprop(lr=0.0001, decay=1e-6)

# Let's train the model using RMSprop
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])



In [0]:
# 上のセルで作ったモデルの概略を表示してみましょう
keras.utils.print_summary(model)

In [0]:
# 学習に先立って調整します
import keras
# Convert class vectors to binary class matrices.
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)


In [0]:
#データの確認
np.min(x_train[0]), np.max(x_train[0])

In [0]:
# データを変換します
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

In [0]:
# 実際に訓練させます。時間がないので，ごく少数のデータだけ使うことにします
history = model.fit(x_train[:20000], y_train[:20000],
                    batch_size=32,
                    epochs=30,
                    verbose=1,
                    validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])


In [0]:
keras.utils.print_summary(model)